# Role: Medical Device Computer
### This notebook acts like an verifier in blockchain. What we do is to fetch the operation from device wallet. And then, we will examine the operation safety to prevent improper operations from hackers. Finally, we will do the action based the operation.

## 1. Instatiate Controller for Verifiers's Agent

In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 7992
WEBHOOK_BASE = ""
ADMIN_URL = "http://medical_device_computer-agent:7991"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic present_proof, printing out the payload. The agent calls to this webhook every time it receives a present proof protocol message from another agent. 


In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

def proof_handler(payload):
    print("Handle present proof")
    print(payload)

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}

agent_controller.register_listeners([connection_listener,proof_listener], defaults=True)



Subscribing too: connections
Subscribing too: present_proof


### 3. Check connection

In [4]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])
        connection_id = item['connection_id']

EXISTING CONNECTIONS
ConnectionID: 9f6336e4-6627-4834-b796-f2a42f6ca03d Status: active


### 4. Create proof request

In [5]:
#please modidy the corresponding credential id in hospital-issue-device part
hospital_did = 'PEcmuSQ7G3z771J7dse1R5'

# schema_id = 'EuEtnVakYFyBtGFT1nHYtH:2:SSI PyDentity Tutorial:0.0.1' 

print("Request proof of Fullname and Age range from Identity Holder")
#Set some variables

exchange_tracing = False

req_attrs = [
    {"name": "type", "restrictions": [{"issuer_did": hospital_did}]},
    {"name": "dose", "restrictions": [{"issuer_did": hospital_did}]},
    {"name": "speed", "restrictions": [{"issuer_did": hospital_did}]},
    {"name": "volt", "restrictions": [{"issuer_did": hospital_did}]}
]

req_preds = []


indy_proof_request = {
    "name": "Proof of Completion of PyDentity SSI Tutorial",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}


#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

Request proof of Fullname and Age range from Identity Holder


### 5. Go to the_wallet notebook to setup

### 8. Send the proof request to Identity Holder

Identity Holder is identified through the connection_id

In [6]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)


{'created_at': '2023-01-06 05:14:46.531239Z', 'initiator': 'self', 'auto_present': False, 'state': 'request_sent', 'presentation_request': {'name': 'Proof of Completion of PyDentity SSI Tutorial', 'version': '1.0', 'requested_attributes': {'0_type_uuid': {'name': 'type', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_dose_uuid': {'name': 'dose', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_speed_uuid': {'name': 'speed', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_volt_uuid': {'name': 'volt', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}}, 'requested_predicates': {}, 'nonce': '897404293099002789488009'}, 'presentation_exchange_id': 'ebb6014e-f748-4da3-97df-b79ce41b23b6', 'role': 'verifier', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '33992292-05e7-4dd3-8005-38fb5b3414c3', 'request_presentations~attach': [{'@id': 'libindy-request-presentat

### 9. Go to the_wallet notebook to send verify response

### 12. Verify Proof Presentation

This is checking the signatures on the credentials presented against the credential schema and definition id stored and resolvable on the ledger. It is a bit of a big complicated object, so we show the common pattern for breaking it down, checking it's verified and accessing the data that has been presented.

In [7]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

Handle present proof
{'created_at': '2023-01-06 05:14:46.531239Z', 'initiator': 'self', 'auto_present': False, 'state': 'verified', 'verified': 'true', 'presentation_request': {'name': 'Proof of Completion of PyDentity SSI Tutorial', 'version': '1.0', 'requested_attributes': {'0_type_uuid': {'name': 'type', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_dose_uuid': {'name': 'dose', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_speed_uuid': {'name': 'speed', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}, '0_volt_uuid': {'name': 'volt', 'restrictions': [{'issuer_did': 'PEcmuSQ7G3z771J7dse1R5'}]}}, 'requested_predicates': {}, 'nonce': '897404293099002789488009'}, 'presentation_exchange_id': 'ebb6014e-f748-4da3-97df-b79ce41b23b6', 'role': 'verifier', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '33992292-05e7-4dd3-8005-38fb5b3414c3', 'request_presentations~attach'

### 13. Verifying the State

Once verified through the agent framework api, the state should be 'verified', otherwise the presentation is invalid.

In [8]:
print(verify['state'])
print(verify['state'] == 'verified')

verified
True


### 14. Accessing the Revealed Attributes (This is probably talking about how to access the whole data, instead of asking question. Need to be confirmed.)

In [9]:
for (name, val) in verify['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name)
    ## Slightly different for self attested attrs
    print(val)

### 15. Design the whole medical device computer.

In [10]:
operation = {}
for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    name = name[2:-5]
    operation[name] = val['raw']
match operation["type"]:
    case "drug_A":
        if int(operation["dose"])<30:
            print(f"inject drug_A {operation['dose']}ml")
        else:
            print("overdose,please contact the doctor")
        pass
    case "drug_B":
        print("this drug need permission of doctor to inject")
    case "nasogastric tube":
        if int(operation['speed'])<25:
            print(f"adjust the speed to {operation['speed']}")
        else:
            print("the speed is too high")
    case "x-ray":
        print("X-ray is ready")
    case "electrotherapy-devise":
        if int(operation['volt'])>30:
            print()
    case _:
        print("unknown operation,please check")
        pass


inject drug_A 1ml


### **Terminate**

In [ ]:
response = await agent_controller.terminate()
print(response)